In [4]:
import gym
import random
import numpy as np
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

Using Theano backend.
Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'C:\\Users\\tera2\\AppData\\Local\\Temp\\try_flags_f9qy_myb.c:4:19: fatal error: cudnn.h: No such file or directory\r\ncompilation terminated.\r\n'
Preallocating 716/1024 Mb (0.700000) on cuda0
Mapped name None to device cuda0: GeForce GTX 460 (0000:01:00.0)


In [5]:
env = gym.make('MountainCar-v0')
env.reset()
goal_steps = 200
score_requirement = -198
intial_games = 10000

In [6]:
def random_game():
    for step_index in range(goal_steps):
#         env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("observation: {}".format(observation))
        print("reward: {}".format(reward))
        print("done: {}".format(done))
        print("info: {}".format(info))
        if done:
            break
    env.reset()

In [ ]:
random_game()

In [8]:

def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 3)
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            if observation[0] > -0.25:
                reward = 1
            
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1, 0]
                elif data[1] == 0:
                    output = [1, 0, 0]
                elif data[1] == 2:
                    output = [0, 0, 1]
                training_data.append([data[0], output])
        
        env.reset()
    
    print(accepted_scores)
    
    return training_data

In [9]:
training_data = model_data_preparation()

[-172.0, -172.0, -198.0, -170.0, -188.0, -154.0, -198.0, -168.0, -184.0, -180.0, -184.0, -186.0, -168.0, -186.0, -182.0, -190.0, -186.0, -182.0, -172.0, -180.0, -192.0, -198.0, -180.0, -182.0, -176.0, -194.0, -178.0, -194.0, -186.0, -194.0, -188.0, -178.0, -154.0, -178.0, -180.0, -194.0, -188.0, -178.0, -188.0, -174.0, -186.0, -194.0, -178.0, -158.0, -190.0, -168.0, -172.0, -184.0, -174.0, -188.0, -168.0, -184.0, -192.0, -180.0, -176.0, -168.0, -194.0, -176.0, -170.0, -166.0, -182.0, -184.0, -184.0, -182.0, -174.0, -172.0, -186.0, -192.0, -182.0, -184.0, -176.0, -180.0, -182.0, -192.0, -184.0, -178.0, -184.0, -178.0, -170.0, -186.0, -190.0, -172.0, -166.0, -174.0, -172.0, -186.0, -184.0, -174.0, -178.0, -182.0, -198.0, -184.0, -178.0, -172.0, -176.0, -132.0, -194.0, -190.0, -184.0, -170.0, -186.0, -186.0, -184.0, -176.0, -186.0, -170.0, -194.0, -180.0, -176.0, -194.0, -184.0, -166.0, -190.0, -168.0, -192.0, -168.0, -176.0, -174.0, -168.0, -182.0, -158.0, -184.0, -184.0, -164.0, -184.0,

In [10]:
def build_model(input_size, output_size):
        model = Sequential()
        model.add(Dense(128, input_dim=input_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(output_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam())

        return model

In [11]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=3)
    return model

In [12]:

trained_model = train_model(training_data)

Epoch 1/3
44178/44178 [==============================] - 2s 48us/step - loss: 0.2231
Epoch 2/3
44178/44178 [==============================] - 2s 50us/step - loss: 0.2210
Epoch 3/3
44178/44178 [==============================] - 2s 49us/step - loss: 0.2208


In [13]:
model_json = trained_model.to_json()
# Записываем модель в файл
json_file = open("model_AIGame_MC_v2.json", "w")
json_file.write(model_json)
json_file.close()
trained_model.save_weights("model_AIGame_MC_v2.h5")

In [15]:
scores = []
choices = []
for each_game in range(100):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        # Uncomment this line if you want to see how our bot playing
        env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{} choice 2:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices),choices.count(2)/len(choices)))

[-164.0, -92.0, -162.0, -87.0, -173.0, -167.0, -117.0, -98.0, -173.0, -168.0, -96.0, -165.0, -105.0, -91.0, -91.0, -164.0, -90.0, -101.0, -164.0, -95.0, -168.0, -164.0, -106.0, -162.0, -169.0, -164.0, -167.0, -164.0, -200.0, -170.0, -115.0, -163.0, -184.0, -185.0, -105.0, -162.0, -165.0, -112.0, -181.0, -95.0, -129.0, -165.0, -163.0, -175.0, -167.0, -176.0, -98.0, -109.0, -88.0, -164.0, -106.0, -96.0, -162.0, -100.0, -92.0, -167.0, -162.0, -88.0, -200.0, -102.0, -164.0, -185.0, -89.0, -167.0, -108.0, -162.0, -111.0, -170.0, -92.0, -171.0, -164.0, -104.0, -162.0, -186.0, -170.0, -164.0, -179.0, -89.0, -176.0, -164.0, -169.0, -168.0, -192.0, -183.0, -88.0, -163.0, -168.0, -166.0, -89.0, -96.0, -92.0, -104.0, -164.0, -87.0, -195.0, -91.0, -87.0, -119.0, -164.0, -185.0]
Average Score: -141.2
choice 1:0.032577903682719546  choice 0:0.4296742209631728 choice 2:0.5377478753541076
